<a href="https://colab.research.google.com/github/slurya/experimental/blob/main/trailhead_tf_daisy_metrics_xmao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pandas as pd
import numpy as np
from datetime import datetime
from openpyxl import load_workbook
import os


In [ ]:
import re
def parse_daisy_answer(answer):
    result = re.search("(true|false)$", str(answer).strip().lower())
    result_ = re.search("(true|false)", str(answer).strip().lower())
    if result:
      if result.group() == "true":
          return True
      elif result.group() == "false":
          return False
      else:
          return None
    elif result_:
      if result_.group() == "true":
          return True
      elif result_.group() == "false":
          return False
      else:
          return None
    else:
        return answer

def parse_java_result(raw_result):
    if isinstance(raw_result, str):
        pattern = r"'knowledgeSummary'\s*:\s*'(True|False)'"
        result = re.search(pattern, raw_result.replace("\n", ""))

        if result:
            # print(result)
            if result.group(1).strip() == "False":
                return False
            elif result.group(1).strip() == "True":
                return True
            else:
                return None
        else:
            return raw_result
    else:
        return raw_result

def parse_llm_question(question):
    question = question.replace("Please help answer with only true or false as the response: ", "").strip()
    # if not question.startswith("True or false: "):
    #     question = "True or false: " + question
    return question

def proccess_groud_truth_question(question):
    return question.strip()


def get_non_bool_answer(joined_df):
    resp_columns = [daisy_resp_col]
    format_issue = {}
    can_not_answer = {}
    for resp_from in resp_columns:
        if resp_from not in joined_df.columns:
            continue
        format_issue[resp_from] = []
        can_not_answer[resp_from] = []
        for agent_resp in joined_df[resp_from].to_list():
            if agent_resp not in (True, False):
                if "sorry" in agent_resp.lower():
                    can_not_answer[resp_from].append(agent_resp)
                else:
                    format_issue[resp_from].append(agent_resp)
    return format_issue, can_not_answer


def calculate_metric_agreement_yes_no(auto, human):
   human = np.asarray([str(label).strip().lower() for label in human])
   n_samples = len(human)
   for label in human:  # sanity check
       if label not in ['true', 'false']:
           print('WRONG LABEL!', label)

   auto = np.asarray([str(v).strip().lower()  for v in auto])
   n_wrong_label = 0
   for label in auto:
       if label not in ['true', 'false']:  # sanity check
           print('NONE LABEL!', label)
           n_wrong_label += 1
   print(f"{n_wrong_label} NONE labels")
   human = human[auto != None]
   auto = auto[auto != None]

   agreement = sum(human == auto) / n_samples
   precision = sum((human == 'true') & (auto == 'true')) / sum(auto == 'true')
   recall = sum((human == 'true') & (auto == 'true')) / sum(human == 'true')

   return agreement, precision, recall


In [ ]:
# groud_truth_path
ground_truth = "/content/drive/MyDrive/auto_evaluator/daisy_planner/dashboard/Data/TrailHead/filtered_ground_truth.csv"
# daisy data path
daisy_data = "/content/drive/MyDrive/auto_evaluator/daisy_planner/dashboard/Data/TrailHead/trailhead_tf_daisy_planner_20250413_224616.csv"

daisy_resp_col = "daisy_response"

ground_truth_df = pd.read_csv(ground_truth)
daisy_result_df = pd.read_csv(daisy_data)
# daisy_result_df = pd.read_excel(open(daisy_data, "rb"), sheet_name="tf_376_04_10")

# remove unused columns
ground_truth_df = ground_truth_df[["question", "ground_truth"]]
daisy_result_df = daisy_result_df[["query", "res"]].rename(columns={"query": "question"})

In [ ]:
import json
def parse_planner_raw_response(raw_response):
    try:
        return json.loads(raw_response)
    except Exception as e:
        print(f"Error parsing raw response: {e}")
        print(raw_response)

In [ ]:
daisy_result_df["json_res"] = daisy_result_df["res"].map(parse_planner_raw_response)

In [ ]:
daisy_result_df[daisy_resp_col] = daisy_result_df["json_res"].map(lambda x: x["payload"]["message"])

In [ ]:
# parse the answer
# daisy_result_df["question"] = daisy_result_df["question"].map(parse_llm_question)
daisy_result_df[daisy_resp_col] = daisy_result_df[daisy_resp_col].map(parse_daisy_answer)

ground_truth_df["question"] = ground_truth_df['question'].map(proccess_groud_truth_question)


Results from Daisy as shared in here - https://docs.google.com/spreadsheets/d/1jLLW7vuMkadiryxbIj9srRo3yYImw0WDHxD37rTF4AI/edit?gid=779458238#gid=779458238

In [ ]:
daisy_result_df

,question,res,json_res,daisy_response
0,True or false: When Data Processing Engine gen...,"{""id"": ""a7e98c11-8371-498e-b2b3-b1b01e4062e3"",...","{'id': 'a7e98c11-8371-498e-b2b3-b1b01e4062e3',...",True
1,True or false: You can integrate identity prov...,"{""payload"": {""messageType"": ""InformResponseMes...",{'payload': {'messageType': 'InformResponseMes...,"Hi, this is the default response from the pyth..."
2,True or false: Cloud computing enables users t...,"{""id"": ""be9d533d-7c62-4ac6-ae32-02549c035f5a"",...","{'id': 'be9d533d-7c62-4ac6-ae32-02549c035f5a',...",True
3,True or false: While many B2C customers choose...,"{""payload"": {""messageType"": ""InformResponseMes...",{'payload': {'messageType': 'InformResponseMes...,"Hi, this is the default response from the pyth..."
4,True or false: Design systems require designer...,"{""id"": ""9dc69faa-ba1f-4bb6-9dce-76f6537f4af1"",...","{'id': '9dc69faa-ba1f-4bb6-9dce-76f6537f4af1',...",False
...,...,...,...,...
95,True or false: exit criteria defines how much ...,"{""id"": ""a28789dc-e42a-49f1-8f39-b0e45e5c50dd"",...","{'id': 'a28789dc-e42a-49f1-8f39-b0e45e5c50dd',...",True
96,True or false: To access content that you’ve s...,"{""payload"": {""messageType"": ""InformResponseMes...",{'payload': {'messageType': 'InformResponseMes...,"Hi, this is the default response from the pyth..."
97,True or false: You should pass SubscriberKey v...,"{""payload"": {""messageType"": ""InformResponseMes...",{'payload': {'messageType': 'InformResponseMes...,"Hi, this is the default response from the pyth..."
98,True or false: CSP digital transformation focu...,"{""id"": ""5c37ebea-f6a4-4245-b0ad-1fcffebc20a7"",...","{'id': '5c37ebea-f6a4-4245-b0ad-1fcffebc20a7',...",False


Left joining with filtered ground truth list (filtered_ground_truth.csv: https://drive.google.com/drive/folders/17fQfJ-Me3p--kshvsvQ3fv4PLbOLvg6r) on question field

In [ ]:
joined_df = ground_truth_df.merge(daisy_result_df, on="question", how="left")
joined_df

,question,ground_truth,res,json_res,daisy_response
0,True or false: When Data Processing Engine gen...,True,"{""id"": ""a7e98c11-8371-498e-b2b3-b1b01e4062e3"",...","{'id': 'a7e98c11-8371-498e-b2b3-b1b01e4062e3',...",True
1,True or false: You can integrate identity prov...,False,"{""payload"": {""messageType"": ""InformResponseMes...",{'payload': {'messageType': 'InformResponseMes...,"Hi, this is the default response from the pyth..."
2,True or false: Cloud computing enables users t...,True,"{""id"": ""be9d533d-7c62-4ac6-ae32-02549c035f5a"",...","{'id': 'be9d533d-7c62-4ac6-ae32-02549c035f5a',...",True
3,True or false: While many B2C customers choose...,True,"{""payload"": {""messageType"": ""InformResponseMes...",{'payload': {'messageType': 'InformResponseMes...,"Hi, this is the default response from the pyth..."
4,True or false: Design systems require designer...,False,"{""id"": ""9dc69faa-ba1f-4bb6-9dce-76f6537f4af1"",...","{'id': '9dc69faa-ba1f-4bb6-9dce-76f6537f4af1',...",False
...,...,...,...,...,...
371,True or false: A history of emails sent throug...,True,NaN,NaN,NaN
372,True or false: A sales manager must select a r...,True,NaN,NaN,NaN
373,True or false: Salesforce Flow for Service hel...,True,NaN,NaN,NaN
374,True or false: A product launch is a moment in...,True,NaN,NaN,NaN


**Total number of questions in the provided filtered ground truth but not present in Daisy output:** 6

In [ ]:
ground_truth_df_nan = joined_df[joined_df[daisy_resp_col].isna()]
length = len(ground_truth_df_nan)
print("Total number of questions in the provided filtered ground truth but not present in Daisy output:", length)
ground_truth_df_nan

Total number of questions in the provided filtered ground truth but not present in Daisy output: 276


,question,ground_truth,res,json_res,daisy_response
100,True or false: You can use Salesforce Maps on ...,True,NaN,NaN,NaN
101,True or false: You need to make sure locales a...,True,NaN,NaN,NaN
102,True or false: All email sends in Marketing Cl...,False,NaN,NaN,NaN
103,True or false: You should store credit card nu...,False,NaN,NaN,NaN
104,True or false: A unified individual profile is...,True,NaN,NaN,NaN
...,...,...,...,...,...
371,True or false: A history of emails sent throug...,True,NaN,NaN,NaN
372,True or false: A sales manager must select a r...,True,NaN,NaN,NaN
373,True or false: Salesforce Flow for Service hel...,True,NaN,NaN,NaN
374,True or false: A product launch is a moment in...,True,NaN,NaN,NaN


In [ ]:
joined_df = joined_df.dropna(subset=[daisy_resp_col])
joined_df

,question,ground_truth,res,json_res,daisy_response
0,True or false: When Data Processing Engine gen...,True,"{""id"": ""a7e98c11-8371-498e-b2b3-b1b01e4062e3"",...","{'id': 'a7e98c11-8371-498e-b2b3-b1b01e4062e3',...",True
1,True or false: You can integrate identity prov...,False,"{""payload"": {""messageType"": ""InformResponseMes...",{'payload': {'messageType': 'InformResponseMes...,"Hi, this is the default response from the pyth..."
2,True or false: Cloud computing enables users t...,True,"{""id"": ""be9d533d-7c62-4ac6-ae32-02549c035f5a"",...","{'id': 'be9d533d-7c62-4ac6-ae32-02549c035f5a',...",True
3,True or false: While many B2C customers choose...,True,"{""payload"": {""messageType"": ""InformResponseMes...",{'payload': {'messageType': 'InformResponseMes...,"Hi, this is the default response from the pyth..."
4,True or false: Design systems require designer...,False,"{""id"": ""9dc69faa-ba1f-4bb6-9dce-76f6537f4af1"",...","{'id': '9dc69faa-ba1f-4bb6-9dce-76f6537f4af1',...",False
...,...,...,...,...,...
95,True or false: exit criteria defines how much ...,True,"{""id"": ""a28789dc-e42a-49f1-8f39-b0e45e5c50dd"",...","{'id': 'a28789dc-e42a-49f1-8f39-b0e45e5c50dd',...",True
96,True or false: To access content that you’ve s...,False,"{""payload"": {""messageType"": ""InformResponseMes...",{'payload': {'messageType': 'InformResponseMes...,"Hi, this is the default response from the pyth..."
97,True or false: You should pass SubscriberKey v...,False,"{""payload"": {""messageType"": ""InformResponseMes...",{'payload': {'messageType': 'InformResponseMes...,"Hi, this is the default response from the pyth..."
98,True or false: CSP digital transformation focu...,False,"{""id"": ""5c37ebea-f6a4-4245-b0ad-1fcffebc20a7"",...","{'id': '5c37ebea-f6a4-4245-b0ad-1fcffebc20a7',...",False


**Answers mismatched between Daisy and Ground truth: 37**




In [ ]:
daisy_df_filtered = joined_df[joined_df[daisy_resp_col] != joined_df['ground_truth']]
length = len(daisy_df_filtered)
print("Length of daisy filtered based on mismatched results:", length)
daisy_df_filtered

Length of daisy filtered based on mismatched results: 49


,question,ground_truth,res,json_res,daisy_response
1,True or false: You can integrate identity prov...,False,"{""payload"": {""messageType"": ""InformResponseMes...",{'payload': {'messageType': 'InformResponseMes...,"Hi, this is the default response from the pyth..."
2,True or false: Cloud computing enables users t...,True,"{""id"": ""be9d533d-7c62-4ac6-ae32-02549c035f5a"",...","{'id': 'be9d533d-7c62-4ac6-ae32-02549c035f5a',...",\n\nTo answer your question about whether clou...
3,True or false: While many B2C customers choose...,True,"{""payload"": {""messageType"": ""InformResponseMes...",{'payload': {'messageType': 'InformResponseMes...,"Hi, this is the default response from the pyth..."
5,True or false: The Loyalty Program Performance...,False,"{""id"": ""45db022e-52d9-4c93-986c-af2b25667bcd"",...","{'id': '45db022e-52d9-4c93-986c-af2b25667bcd',...",True
8,True or false: Building a great dashboard is a...,True,"{""payload"": {""messageType"": ""InformResponseMes...",{'payload': {'messageType': 'InformResponseMes...,"Hi, this is the default response from the pyth..."
9,True or false: The benefit of using static and...,True,"{""id"": ""0c387d3e-5330-44b0-b93c-00d2d6b28ee3"",...","{'id': '0c387d3e-5330-44b0-b93c-00d2d6b28ee3',...",\n\nTo determine if the statement about the be...
10,"True or false: To align your key stakeholders,...",False,"{""id"": ""2e6da8ca-86f9-412a-b6b5-638ab3bd6baa"",...","{'id': '2e6da8ca-86f9-412a-b6b5-638ab3bd6baa',...",\n\nTo answer your question about whether you ...
12,True or false: It’s important to factor in an ...,False,"{""id"": ""67935140-2ddf-435f-80b8-c17ec93f886d"",...","{'id': '67935140-2ddf-435f-80b8-c17ec93f886d',...",True
17,True or false: You must use a different conten...,False,"{""id"": ""bca96fda-78d7-485f-8d2b-43de7d6d72ad"",...","{'id': 'bca96fda-78d7-485f-8d2b-43de7d6d72ad',...",\n\nTo answer your question about whether you ...
20,True or false: The AWS CLI is helpful when aut...,True,"{""payload"": {""messageType"": ""InformResponseMes...",{'payload': {'messageType': 'InformResponseMes...,"Hi, this is the default response from the pyth..."


In [ ]:
mismatch_df_in_customized_topic = daisy_df_filtered[daisy_df_filtered['topic'].str.startswith('p_')]
print(mismatch_df_in_customized_topic.shape)
mismatch_df_in_customized_topic

KeyError: 'topic'

In [ ]:
# could be RAG action / knowledge action issue
mismatch_df_in_customized_topic[mismatch_df_in_customized_topic[daisy_resp_col].isin([True, False])].shape

(26, 4)

In [ ]:
mismatch_df_non_customized_topic = daisy_df_filtered[~daisy_df_filtered['topic'].str.startswith('p_')]
print(mismatch_df_non_customized_topic.shape)
mismatch_df_non_customized_topic

(77, 4)


,question,ground_truth,daisy_response,topic
7,True or false: New Salesforce TM Forum APIs ar...,True,\n\n\n\n\n\nIt looks like there was an issue r...,Prompt_Injection
12,True or false: It’s important to factor in an ...,False,True,Prompt_Injection
15,True or false: If you add or update the produc...,False,\n\n\n\n\n\nIt looks like I'm having trouble r...,Prompt_Injection
27,True or false: Job lists are generated for bot...,False,\n\n\n\n\n\nIt seems there's an issue retrievi...,Prompt_Injection
30,True or false: Your D2C channel provides acces...,True,\n\n\n\n\n\nI'm currently unable to retrieve t...,Prompt_Injection
...,...,...,...,...
349,True or false: The Insurance Distributor Perfo...,False,\n\n\n\n\n\nI'm currently unable to retrieve t...,Prompt_Injection
358,True or false: OFM includes built-in reports t...,True,\n\n\n\n\n\nI'm currently unable to retrieve t...,Prompt_Injection
359,"True or false: When you execute test plans, th...",False,\n\n\n\n\n\nIt looks like I couldn't retrieve ...,Prompt_Injection
360,True or false: The administrative account mode...,False,True,Prompt_Injection


In [ ]:
daisy_df_filtered['topic'].value_counts()

,count
topic,
Prompt_Injection,72
p_16jSB000000M5U5_Sales_Team_Efficiency,25
p_16jSB000000M5U5_Salesforce_Admin_Basics,15
p_16jSB000000M5U5_Business_User_Training,4
p_16jSB000000M5U5_Salesforce_Developer_Essentials,3
p_16jSB000000M5U5_CRM_Customization,3
Default_Category,3
Inappropriate_Content,2


In [ ]:
joined_df['topic'].value_counts()

,count
topic,
Prompt_Injection,181
p_16jSB000000M5U5_Sales_Team_Efficiency,59
p_16jSB000000M5U5_Salesforce_Admin_Basics,56
p_16jSB000000M5U5_Business_User_Training,22
p_16jSB000000M5U5_Salesforce_Developer_Essentials,21
Inappropriate_Content,8
p_16jSB000000M5U5_CRM_Customization,7
Default_Category,6


In [ ]:
daisy_df_filtered.to_csv('/content/drive/MyDrive/auto_evaluator/daisy_planner/dashboard/Data/TrailHead/04_03_2025/daisy_incorrect_answers.csv')

In [ ]:
# save the joined df
joined_df.to_csv(f"/content/drive/MyDrive/auto_evaluator/daisy_planner/dashboard/Data/TrailHead/04_03_2025/Binary_joined_{datetime.today()}.csv")

In [ ]:
format_issue, can_not_answer = get_non_bool_answer(joined_df)

In [ ]:
daisy_filter_df = joined_df[joined_df[daisy_resp_col].isin([True, False])]
daisy_agreement, daisy_precision, daisy_recall = calculate_metric_agreement_yes_no(daisy_filter_df[daisy_resp_col].to_list(), daisy_filter_df["ground_truth"].to_list())

0 NONE labels


In [ ]:
today = datetime.today().strftime("%Y/%m/%d")
result_data = []

result_data.append({
    "Dataset": "TrailHead T/F",
    "Eval Date": today,
    "Data size": len(joined_df),
    "Planner version": "Daisy",
    # "Correct format response": len(daisy_filter_df),
    "Bad output format": len(format_issue[daisy_resp_col]),
    "can't answer": len(can_not_answer[daisy_resp_col]),
    "Coverage": f"{1 - (len(format_issue[daisy_resp_col]) + len(can_not_answer[daisy_resp_col])) / len(joined_df):.2%}" ,
    "Accuracy": f"{daisy_agreement:.2%}" ,
    "Precision": f"{daisy_precision:.2%}" ,
    "Recall":  f"{daisy_recall:.2%}" ,
    "Comment": str({"format issue": format_issue[daisy_resp_col], "can't answer": can_not_answer[daisy_resp_col]})
})

result_df = pd.DataFrame(result_data)

In [ ]:
result_df

,Dataset,Eval Date,Data size,Planner version,Bad output format,can't answer,Coverage,Accuracy,Precision,Recall,Comment
0,TrailHead T/F,2025/04/14,100,Daisy,21,0,79.00%,88.61%,86.54%,95.74%,"{'format issue': ['Hi, this is the default res..."


In [ ]:
def append_to_excel(df, file_path, sheet_name='Sheet1'):
    file_exists = os.path.exists(file_path)
    mode = 'a' if file_exists else 'w'
    if_sheet_exists = 'overlay' if file_exists else None
    header = False if file_exists else True

    with pd.ExcelWriter(
        file_path,
        engine='openpyxl',
        mode=mode,
        if_sheet_exists=if_sheet_exists
    ) as writer:
        if file_exists:
            # writer.book = load_workbook(file_path)
            if sheet_name in writer.book.sheetnames:
                startrow = writer.book[sheet_name].max_row
            else:
                startrow = 0
                header = True
        else:
            startrow = 0

        df.to_excel(
            writer,
            sheet_name=sheet_name,
            startrow=startrow,
            index=False,
            header=header
        )


In [ ]:
today = datetime.today().strftime("%Y-%m-%d")
file_path = f"/content/drive/MyDrive/auto_evaluator/daisy_planner/dashboard/Data/TrailHead/trailhead_tf_eval_{today}.xlsx"

append_to_excel(pd.DataFrame(result_data), file_path)